In [15]:
import os
from bs4 import BeautifulSoup
import re
import pandas as pd

ta_dir_name = 'tennis_abstract'

In [12]:
def get_ta_player_matches(ta_name):
    html_path = f'{ta_dir_name}/{ta_name}.html'
    assert os.path.exists(html_path)
    with open(html_path, 'r') as file:
        html_content = file.read()
    soup = BeautifulSoup(html_content, "html.parser")
    script_tags = soup.find_all("script")
    matchmx = None
    for script in script_tags:
        if script.string:  # Ensure the script tag contains JavaScript code
            match = re.search(r'var\s+matchmx\s*=\s*(\[.*?\]);', script.string, re.DOTALL)
            if match:
                # Extract the JavaScript array or object
                ochoices_str = match.group(1)
                # Convert the JavaScript array/object to a Python object
                matchmx = eval(ochoices_str)
    return matchmx

matchmx = get_ta_player_matches('CarlosAlcaraz')


In [7]:
hdrsServe = ['Date', 'Tournament', 'Surface', 'Rd', 'Rk', 'vRk', '', 'Score', 'More', 'DR', 'A%', 'DF%', '1stIn', '1st%', '2nd%', 'BPSvd', 'Time']
hdrsReturn = ['Date', 'Tournament', 'Surface', 'Rd', 'Rk', 'vRk', '', 'Score', 'More', 'DR', 'TPW', 'RPW', 'vA%', 'v1st%', 'v2nd%', 'BPCnv', 'Time']
hdrsRaw = ['Date', 'Tournament', 'Surface', 'Rd', 'Rk', 'vRk', '', 'Score', 'More', 'TP', 'Aces', 'DFs', 'SP', '1SP', '2SP', 'vA', 'Time']
match_example = ["20250113", "Australian Open", "Hard", "G", "L", "3", "3", "", "QF", "4-6 6-4 6-3 6-4", "5", "Novak Djokovic", "7", "7", "", "R", "19870522", "188", "SRB", "0", "217", "10", "5", "125", "92", "62", "11", "19", "7", "13", "5", "3", "136", "86", "54", "29", "20", "7", "11", "2", "", "", "", "2025-580-222", "", "", "", "104925"]
match_example_inverse = ["20250113", "Australian Open", "Hard", "G", "W", "7", "7", "", "QF", "4-6 6-4 6-3 6-4", "5", "Carlos Alcaraz", "3", "3", "", "R", "20030505", "183", "ESP", "0", "217", "5", "3", "136", "86", "54", "29", "20", "7", "11", "10", "5", "125", "92", "62", "11", "19", "7", "13", "2", "", "", "", "2025-580-222", "", "", "", "207989"]
mahac_example = ["20250113", "Australian Open", "Hard", "G", "W", "25", "26", "", "R64", "3-6 7-6(1) 6-7(5) 7-6(4) 6-4", "5", "Reilly Opelka", "170", "", "", "R", "19970828", "211", "USA", "0", "205", "16", "7", "167", "105", "88", "36", "28", "7", "8", "40", "16", "173", "115", "102", "24", "27", "4", "5", "2", "", "", "", "2025-580-181", "", "2", "2", "124187"]
len(hdrsServe), len(hdrsReturn), len(hdrsRaw), len(match_example)

(17, 17, 17, 48)

In [10]:
match_hdrs = [
    'Date', 'Tournament', 'Surface', "DUNNO G", "Result", "Rank", "Seed", "DUNNO ", "Rd", "Score", 
    "Num of sets (i think) 5", "Opponent", "Opponent rank", "Opponent seed", "Dunno ", "Dunno R", 
    "Opponent birthday", "Opponent height cm", "Opponent nationality", "Dunno 0", "Dunno 217", "Aces", 
    "Double Faults", "Total service points", "Total first serve points", "first serves won", "second serves won", 
    "service_games", "break_points_saved", "break_points_faced", 
    "oaces","odfs", "oTotal service points", "oTotal first serve points", "ofirst serves won", "osecond serves won", 
    "oservice_games", "obreak_points_saved", "obreak_points_faced", "obackhand_arms", "", "", "", "2025-580-222", "", "", "", "oforeign_key"]
matchhead = ["date","tourn","surf","level","wl","rank","seed","entry","round",
                 "score","max","opp","orank","oseed","oentry","ohand","obday",
                 "oht","ocountry","oactive","time","aces","dfs","pts","firsts","fwon",
                 "swon",'games',"saved","chances","oaces","odfs","opts","ofirsts",
                 "ofwon","oswon",'ogames',"osaved","ochances", "obackhand", "chartlink",
                 "pslink","whserver","matchid","wh","roundnum","matchnum"]
matchhead_verbose = ["date","tourn","surf","level","win/loss","rank","seed","entry","round",
                 "score","max_num_of_sets","opp","orank","oseed","oentry","ohand","obday",
                 "oheight","ocountry","oactive","time_minutes","aces","dfs","service_pts","first_serves_in","first_serves_won",
                 "second_serves_won",'service_games',"break_points_saved","break_points_faced","oaces","odfs","oservice_pts","ofirst_serves_in","ofirst_serves_won",
                 "osecond_serves_won",'oservice_games',"obreak_points_saved","obreak_points_faced", "obackhand", "chartlink",
                 "pslink","whserver","matchid","wh","roundnum","matchnum", "oforeign_key"]
levels = {
    'G': 'Grand Slam',
    'M': 'Masters',
    'A': 'ATP',
    'D': 'Davis Cup',
    'Q': 'Qualifying'
    # ...
}
len(matchhead)

47

In [17]:
len(matchmx)
match_df = pd.DataFrame(matchmx, columns=matchhead_verbose)
match_df.iloc[0]


date                          20250113
tourn                  Australian Open
surf                              Hard
level                                G
win/loss                             L
rank                                 3
seed                                 3
entry                                 
round                               QF
score                  4-6 6-4 6-3 6-4
max_num_of_sets                      5
opp                     Novak Djokovic
orank                                7
oseed                                7
oentry                                
ohand                                R
obday                         19870522
oheight                            188
ocountry                           SRB
oactive                              0
time_minutes                       217
aces                                10
dfs                                  5
service_pts                        125
first_serves_in                     92
first_serves_won         